In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import math

#  error_type 1
def checkHttpStatusCode(r, url):
    
    if r.status_code != requests.codes.ok:
        print("個別店ID：", Izakaya.personal_store_id)
        print("　指定したURLでスクレイピングに失敗しました。")
        print("　指定したURL：", url)
        return False

    return True

#  error_type 2
def checkSearchStoreCount(search_store_count, url):
    
    if search_store_count == 0:
        print("指定したURLで居酒屋の件数は0件でした。")
        print("指定したURL：", url)
        return False

    return True

#  error_type 3
def checkExistenceDrinkInfo(drink_info_list, url):
    
    if len(drink_info_list) == 0 :
        print("個別店ID：", Izakaya.personal_store_id)
        print("　指定したURLで居酒屋のドリンクは掲載されていませんでした。")
        print("　指定したURL：", url)
        return False
        
    for drink_info in drink_info_list :
        
        if "ドリンク" in drink_info.find('span') :
            print("個別店ID：", Izakaya.personal_store_id)
            print("　指定したURLで居酒屋のドリンクは掲載されていませんでした。")
            print("　指定したURL：", url)
            return False
        
    return True

#  error_type 4
def checkExistenceSearchWords(scraping_search_words_count, url):
    
    if scraping_search_words_count == 0:
        print("個別店ID：", Izakaya.personal_store_id)
        print("　指定したURLで検索ワードのドリンクは掲載されていませんでした。")
        print("　指定したURL：", url)
        return True
        
    return False

def addStoreInfoRecord(store_id
                     , store_name
                     , store_genre
                     , drink_name
                     , drink_price
                     , store_distance
                     , store_budget
                     , store_evaluation
                     , store_url
                     , error_flag
                     , error_type) :
    
    se = pd.Series([store_id
                  , store_name
                  , store_genre
                  , drink_name
                  , drink_price
                  , store_distance
                  , store_budget
                  , store_evaluation
                  , store_url
                  , error_flag
                  , error_type]
                  , Izakaya.columns_izakaya) # 行を作成
    
    Izakaya.df_izakaya = Izakaya.df_izakaya.append(se, Izakaya.columns_izakaya) # データフレームに行を追加

def getAlcoholInfo(personal_store_url, personal_store_name, personal_store_distance, personal_store_genre):
    
    scraping_search_words_count = 0
    
    # 個別店URL + ドリンクタグ
    personal_store_url_add_drink_tag = personal_store_url + "dtlmenu/drink/"
    r = requests.get(personal_store_url_add_drink_tag)
    if checkHttpStatusCode(r, personal_store_url_add_drink_tag):
    
        # 個別店URL + ドリンクタグのBeautifulSoupオブジェクトを生成
        soup = BeautifulSoup(r.content, 'html.parser')
        
        # ドリンクタグのドリンク情報を取得
        drink_tag_drink_info_list = soup.find_all('div', class_='rstdtl-menu-lst__info-inner') 

        # カラム：個別店ID、個別店名、個別店URL、個別店距離、個別店ジャンル、個別店予算、個別店評価
        personal_store_id = Izakaya.personal_store_id
        personal_store_name = personal_store_name
        personal_store_url = personal_store_url
        personal_store_distance = personal_store_distance
        personal_store_genre = personal_store_genre
        personal_store_budget = soup.find('p', class_='rdheader-budget__icon rdheader-budget__icon--dinner').find('span').text.strip()
        personal_store_evaluation = soup.find('span', class_='rdheader-rating__score-val-dtl').text.strip()

        # ドリンクタグのドリンク情報から指定ワードのドリンク名・価格を取得する
        for serach_word in Izakaya.search_words:
            for i in range(len(drink_tag_drink_info_list)):
                if serach_word in (drink_tag_drink_info_list[i].p.text.strip()):
                    if len(drink_tag_drink_info_list[i].select('p')) == 2:
                        scraping_search_words_count += 1
                        personal_store_drink_name = drink_tag_drink_info_list[i].select('p')[0].text.strip()
                        personal_store_drink_price = drink_tag_drink_info_list[i].select('p')[1].text.strip()
                        addStoreInfoRecord(personal_store_id
                                         , personal_store_name
                                         , personal_store_genre
                                         , personal_store_drink_name
                                         , personal_store_drink_price  
                                         , personal_store_distance
                                         , personal_store_budget
                                         , personal_store_evaluation
                                         , personal_store_url
                                         , 0
                                         , ""
                                           )
                        
    # 検索ワードがドリンクタグにない場合
    if checkExistenceSearchWords(scraping_search_words_count, personal_store_url_add_drink_tag):
        
        addStoreInfoRecord(personal_store_id
                         , personal_store_name
                         , personal_store_genre
                         , '' #personal_store_drink_name
                         , '' #personal_store_drink_price  
                         , personal_store_distance
                         , personal_store_budget
                         , personal_store_evaluation
                         , personal_store_url
                         , 1
                         , '4'
                           )
                        
    else:
        
        # 個別店URL + ドリンクタグが不正
        return
        
def getPersonalStoreInfo(r, search_store_count):
    
    # 検索エリアURL + ページ数のBeautifulSoupオブジェクトを生成
    soup = BeautifulSoup(r.content, 'html.parser')
    personal_store_info_list = soup.find_all("h4",class_='list-rst__rst-name')
    
    for i, personal_store_info in enumerate(personal_store_info_list[:search_store_count]):
        
        Izakaya.personal_store_id += 1
            
        # カラム：個別店名
        personal_store_name = personal_store_info.find('a').text
        
        # カラム：個別店URL
        personal_store_url = personal_store_info.find('a').get(('href'))
        
        # 個別店URLのHTTP情報を取得
        personal_store_r = requests.get(personal_store_url)
        if checkHttpStatusCode(personal_store_r, personal_store_url):
        
            # 個別店URLのBeautifulSoupオブジェクトを生成
            personal_store_soup = BeautifulSoup(personal_store_r.content, 'html.parser')
                
            # 個別店URLのメニュータグの情報を取得
            personal_store_menu_tag_info_list = (personal_store_soup.find('li'
                                               , id = "rdnavi-menu"
                                               , class_ = "rstdtl-navi__list-item js-rstdtl-navi-list-item").find_all('div', class_ = "rstdtl-navi__sub-nolink"))
                    
            # カラム：個別店の距離とジャンル
            personal_store_distance_and_genre = personal_store_info.find('span').text 
            personal_store_distance = personal_store_distance_and_genre[0:personal_store_distance_and_genre.find('/')].strip()
            personal_store_genre = personal_store_distance_and_genre[personal_store_distance_and_genre.find('/')+1:len(personal_store_distance_and_genre)].strip()

            # 個別店URLのメニュータグにドリンクの情報がない場合、処理を終了する
            if checkExistenceDrinkInfo(personal_store_menu_tag_info_list, personal_store_url):
                        
                getAlcoholInfo(personal_store_url, personal_store_name, personal_store_distance, personal_store_genre)
                
            # 個別店URLのメニュータグにドリンクの情報がない
            else :
                        
                addStoreInfoRecord(Izakaya.personal_store_id
                                 , personal_store_name
                                 , personal_store_genre
                                 , '' #drink_name
                                 , '' #drink_price
                                 , personal_store_distance
                                 , '' #store_budget
                                 , '' #store_evaluation
                                 , personal_store_url
                                 , '1'
                                 , '3')
                        
        # 個別店URLのHTTP情報が不正
        else :
                    
            addStoreInfoRecord(Izakaya.personal_store_id
                             , personal_store_name
                             , '' #personal_store_genre
                             , '' #drink_name
                             , '' #drink_price
                             , '' #personal_store_distance
                             , '' #store_budget
                             , '' #store_evaluation
                             , personal_store_url
                             , '1'
                             , '1')

def getAreaStoreInfo(search_area_url, search_store_count):

    # 検索店数が0の場合、処理を終了
    if search_store_count == 0 :
        print("指定した検索店数が0件です。")
        return False
    
    # 検索エリアURLのHTTP情報を取得
    r = requests.get(search_area_url)
    if checkHttpStatusCode(r, search_area_url):
            
        # 検索エリアURLのBeautifulSoupオブジェクトを生成
        soup = BeautifulSoup(r.content, 'html.parser')
        
        # 検索エリアURLのスクレイピング店数取得結果が0の場合、処理を終了
        scraping_store_count_get_result = int(soup.find('span', class_='list-condition__count').text.strip())
        if checkSearchStoreCount(scraping_store_count_get_result, search_area_url) :

            # 検索店数がスクレイピング店数取得結果より多い場合、
            # 検索店数をスクレイピング店数取得結果に更新
            # 繰り返しページ数を取得
            if search_store_count > scraping_store_count_get_result:    
                Izakaya.search_store_count = scraping_store_count_get_result
                loop_page_count = math.ceil( Izakaya.search_store_count / Izakaya.separator_page_count )
            else:
                loop_page_count = math.ceil( search_store_count / Izakaya.separator_page_count )

            for page_count in range(loop_page_count):
                
                # 検索エリアURL + ページ数
                search_area_url_add_page_count = search_area_url + str(page_count + 1) + '/'
                r = requests.get(search_area_url_add_page_count)
                if checkHttpStatusCode(r, search_area_url_add_page_count):
                    
                    # 検索店数の更新
                    if page_count == loop_page_count - 1:
                        search_store_count = Izakaya.search_store_count - Izakaya.separator_page_count * page_count
                    else:
                        search_store_count = 20
                    
                    getPersonalStoreInfo(r, search_store_count)
                    
                # 検索エリアURL + ページ数のHTTP情報が不正
                else:
                    return False
            
        # 検索エリアURLのスクレイピング店数取得結果が0
        else:
            return False
    
    # 検索エリアURLのHTTP情報が不正
    else:
        return False
    
    return True
        
class Izakaya:
    
    columns_izakaya = ['store_id'
                     , 'store_name'
                     , 'store_genre'
                     , 'drink_name'
                     , 'drink_price'
                     , 'store_distance'
                     , 'store_budget'
                     , 'store_evaluation'
                     , 'store_url'
                     , 'error_flag'
                     , 'error_type']
    
    df_izakaya = pd.DataFrame(columns=columns_izakaya)
    
    # 区切りページ数
    separator_page_count = 20
    
    # 検索エリアURL
    search_area_url = "https://tabelog.com/chiba/A1202/A120202/R9599/rstLst/izakaya/"

    # 検索ワード
    search_words = ["ビール", "モルツ", "ドライ", "ラガー", "黒ラベル"]

    # 検索店数
    search_store_count = 3
    
    # カラム：個別店ID
    personal_store_id = 0

    def __init__(self, area_store_url, search_store_count):
        
        if getAreaStoreInfo(area_store_url, search_store_count):
            display(Izakaya.df_izakaya)
            # display(Izakaya.df_izakaya.sort_values(by='drink_price'))
            Izakaya.df_izakaya.to_csv("./Izakaya_MinamiGyoutoku.csv", encoding='utf_8_sig')
        

# アプリ「Izakaya」実行
Izakaya(Izakaya.search_area_url, Izakaya.search_store_count)


,store_id,store_name,store_genre,drink_name,drink_price,store_distance,store_budget,store_evaluation,store_url,error_flag,error_type
0,1,とり鉄 南行徳店,居酒屋、うどん、焼鳥,アサヒスーパードライ,290円,南行徳駅 41m,"￥2,000～￥2,999",3.03,https://tabelog.com/chiba/A1202/A120202/12001341/,0,
1,1,とり鉄 南行徳店,居酒屋、うどん、焼鳥,アサヒドライゼロ 瓶,390円,南行徳駅 41m,"￥2,000～￥2,999",3.03,https://tabelog.com/chiba/A1202/A120202/12001341/,0,
2,2,大衆ホルモン タツ屋 南行徳店,居酒屋、ホルモン、焼肉,一番搾り生ビール(中),500円,南行徳駅 90m,"￥2,000～￥2,999",3.12,https://tabelog.com/chiba/A1202/A120202/12025758/,0,
3,2,大衆ホルモン タツ屋 南行徳店,居酒屋、ホルモン、焼肉,ビール(大瓶),500円,南行徳駅 90m,"￥2,000～￥2,999",3.12,https://tabelog.com/chiba/A1202/A120202/12025758/,0,
4,2,大衆ホルモン タツ屋 南行徳店,居酒屋、ホルモン、焼肉,ノンアルコールビールテイスト飲料 キリンゼロイチ,360円,南行徳駅 90m,"￥2,000～￥2,999",3.12,https://tabelog.com/chiba/A1202/A120202/12025758/,0,
5,3,南行徳 彦酉,居酒屋、焼鳥、魚介料理・海鮮料理,アサヒドライゼロ［小瓶］,433円,南行徳駅 232m,"￥3,000～￥3,999",3.09,https://tabelog.com/chiba/A1202/A120202/12033728/,0,
6,3,南行徳 彦酉,居酒屋、焼鳥、魚介料理・海鮮料理,キリンラガー［中瓶］,588円,南行徳駅 232m,"￥3,000～￥3,999",3.09,https://tabelog.com/chiba/A1202/A120202/12033728/,0,
7,3,南行徳 彦酉,居酒屋、焼鳥、魚介料理・海鮮料理,【NEW】サッポロラガー（赤星）［中瓶］,588円,南行徳駅 232m,"￥3,000～￥3,999",3.09,https://tabelog.com/chiba/A1202/A120202/12033728/,0,


In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import math

#  error_type 1
def checkHttpStatusCode(r, url):
    
    if r.status_code != requests.codes.ok:
        print("個別店ID：", Izakaya.personal_store_id)
        print("　指定したURLでスクレイピングに失敗しました。")
        print("　指定したURL：", url)
        return False

    return True

#  error_type 2
def checkSearchStoreCount(search_store_count, url):
    
    if search_store_count == 0:
        print("指定したURLで居酒屋の件数は0件でした。")
        print("指定したURL：", url)
        return False

    return True

#  error_type 3
def checkExistenceDrinkInfo(drink_info_list, url):
    
    if len(drink_info_list) == 0 :
        print("個別店ID：", Izakaya.personal_store_id)
        print("　指定したURLで居酒屋のドリンクは掲載されていませんでした。")
        print("　指定したURL：", url)
        return False
        
    for drink_info in drink_info_list :
        
        if "ドリンク" in drink_info.find('span') :
            print("個別店ID：", Izakaya.personal_store_id)
            print("　指定したURLで居酒屋のドリンクは掲載されていませんでした。")
            print("　指定したURL：", url)
            return False
        
    return True

#  error_type 4
def checkExistenceSearchWords(scraping_search_words_count, url):
    
    if scraping_search_words_count == 0:
        print("個別店ID：", Izakaya.personal_store_id)
        print("　指定したURLで検索ワードのドリンクは掲載されていませんでした。")
        print("　指定したURL：", url)
        return True
        
    return False

def addStoreInfoRecord(store_id
                     , store_name
                     , store_genre
                     , drink_name
                     , drink_price
                     , store_distance
                     , store_budget
                     , store_evaluation
                     , store_url
                     , error_flag
                     , error_type) :
    
    se = pd.Series([store_id
                  , store_name
                  , store_genre
                  , drink_name
                  , drink_price
                  , store_distance
                  , store_budget
                  , store_evaluation
                  , store_url
                  , error_flag
                  , error_type]
                  , Izakaya.columns_izakaya) # 行を作成
    
    Izakaya.df_izakaya = Izakaya.df_izakaya.append(se, Izakaya.columns_izakaya) # データフレームに行を追加

def getAlcoholInfo(personal_store_url, personal_store_name, personal_store_distance, personal_store_genre):
    
    scraping_search_words_count = 0
    
    # 個別店URL + ドリンクタグ
    personal_store_url_add_drink_tag = personal_store_url + "dtlmenu/drink/"
    r = requests.get(personal_store_url_add_drink_tag)
    if checkHttpStatusCode(r, personal_store_url_add_drink_tag):
    
        # 個別店URL + ドリンクタグのBeautifulSoupオブジェクトを生成
        soup = BeautifulSoup(r.content, 'html.parser')
        
        # ドリンクタグのドリンク情報を取得
        drink_tag_drink_info_list = soup.find_all('div', class_='rstdtl-menu-lst__info-inner') 

        # カラム：個別店ID、個別店名、個別店URL、個別店距離、個別店ジャンル、個別店予算、個別店評価
        personal_store_id = Izakaya.personal_store_id
        personal_store_name = personal_store_name
        personal_store_url = personal_store_url
        personal_store_distance = personal_store_distance
        personal_store_genre = personal_store_genre
        personal_store_budget = soup.find('p', class_='rdheader-budget__icon rdheader-budget__icon--dinner').find('span').text.strip()
        personal_store_evaluation = soup.find('span', class_='rdheader-rating__score-val-dtl').text.strip()

        # ドリンクタグのドリンク情報から指定ワードのドリンク名・価格を取得する
        for serach_word in Izakaya.search_words:
            for i in range(len(drink_tag_drink_info_list)):
                if serach_word in (drink_tag_drink_info_list[i].p.text.strip()):
                    if len(drink_tag_drink_info_list[i].select('p')) == 2:
                        scraping_search_words_count += 1
                        personal_store_drink_name = drink_tag_drink_info_list[i].select('p')[0].text.strip()
                        personal_store_drink_price = drink_tag_drink_info_list[i].select('p')[1].text.strip()
                        addStoreInfoRecord(personal_store_id
                                         , personal_store_name
                                         , personal_store_genre
                                         , personal_store_drink_name
                                         , personal_store_drink_price  
                                         , personal_store_distance
                                         , personal_store_budget
                                         , personal_store_evaluation
                                         , personal_store_url
                                         , 0
                                         , ""
                                           )
                        
    # 検索ワードがドリンクタグにない場合
    if checkExistenceSearchWords(scraping_search_words_count, personal_store_url_add_drink_tag):
        
        addStoreInfoRecord(personal_store_id
                         , personal_store_name
                         , personal_store_genre
                         , '' #personal_store_drink_name
                         , '' #personal_store_drink_price  
                         , personal_store_distance
                         , personal_store_budget
                         , personal_store_evaluation
                         , personal_store_url
                         , 1
                         , '4'
                           )
                        
    else:
        
        # 個別店URL + ドリンクタグが不正
        return
        
def getPersonalStoreInfo(r, search_store_count):
    
    # 検索エリアURL + ページ数のBeautifulSoupオブジェクトを生成
    soup = BeautifulSoup(r.content, 'html.parser')
    personal_store_info_list = soup.find_all("h4",class_='list-rst__rst-name')
    
    for i, personal_store_info in enumerate(personal_store_info_list[:search_store_count]):
        
        Izakaya.personal_store_id += 1
            
        # カラム：個別店名
        personal_store_name = personal_store_info.find('a').text
        
        # カラム：個別店URL
        personal_store_url = personal_store_info.find('a').get(('href'))
        
        # 個別店URLのHTTP情報を取得
        personal_store_r = requests.get(personal_store_url)
        if checkHttpStatusCode(personal_store_r, personal_store_url):
        
            # 個別店URLのBeautifulSoupオブジェクトを生成
            personal_store_soup = BeautifulSoup(personal_store_r.content, 'html.parser')
                
            # 個別店URLのメニュータグの情報を取得
            personal_store_menu_tag_info_list = (personal_store_soup.find('li'
                                               , id = "rdnavi-menu"
                                               , class_ = "rstdtl-navi__list-item js-rstdtl-navi-list-item").find_all('div', class_ = "rstdtl-navi__sub-nolink"))
                    
            # カラム：個別店の距離とジャンル
            personal_store_distance_and_genre = personal_store_info.find('span').text 
            personal_store_distance = personal_store_distance_and_genre[0:personal_store_distance_and_genre.find('/')].strip()
            personal_store_genre = personal_store_distance_and_genre[personal_store_distance_and_genre.find('/')+1:len(personal_store_distance_and_genre)].strip()

            # 個別店URLのメニュータグにドリンクの情報がない場合、処理を終了する
            if checkExistenceDrinkInfo(personal_store_menu_tag_info_list, personal_store_url):
                        
                getAlcoholInfo(personal_store_url, personal_store_name, personal_store_distance, personal_store_genre)
                
            # 個別店URLのメニュータグにドリンクの情報がない
            else :
                        
                addStoreInfoRecord(Izakaya.personal_store_id
                                 , personal_store_name
                                 , personal_store_genre
                                 , '' #drink_name
                                 , '' #drink_price
                                 , personal_store_distance
                                 , '' #store_budget
                                 , '' #store_evaluation
                                 , personal_store_url
                                 , '1'
                                 , '3')
                        
        # 個別店URLのHTTP情報が不正
        else :
                    
            addStoreInfoRecord(Izakaya.personal_store_id
                             , personal_store_name
                             , '' #personal_store_genre
                             , '' #drink_name
                             , '' #drink_price
                             , '' #personal_store_distance
                             , '' #store_budget
                             , '' #store_evaluation
                             , personal_store_url
                             , '1'
                             , '1')

def getAreaStoreInfo(search_area_url, search_store_count):

    # 検索店数が0の場合、処理を終了
    if search_store_count == 0 :
        print("指定した検索店数が0件です。")
        return False
    
    # 検索エリアURLのHTTP情報を取得
    r = requests.get(search_area_url)
    if checkHttpStatusCode(r, search_area_url):
            
        # 検索エリアURLのBeautifulSoupオブジェクトを生成
        soup = BeautifulSoup(r.content, 'html.parser')
        
        # 検索エリアURLのスクレイピング店数取得結果が0の場合、処理を終了
        scraping_store_count_get_result = int(soup.find('span', class_='list-condition__count').text.strip())
        if checkSearchStoreCount(scraping_store_count_get_result, search_area_url) :

            # 検索店数がスクレイピング店数取得結果より多い場合、
            # 検索店数をスクレイピング店数取得結果に更新
            # 繰り返しページ数を取得
            if search_store_count > scraping_store_count_get_result:    
                Izakaya.search_store_count = scraping_store_count_get_result
                loop_page_count = math.ceil( Izakaya.search_store_count / Izakaya.separator_page_count )
            else:
                loop_page_count = math.ceil( search_store_count / Izakaya.separator_page_count )

            for page_count in range(loop_page_count):
                
                # 検索エリアURL + ページ数
                search_area_url_add_page_count = search_area_url + str(page_count + 1) + '/'
                r = requests.get(search_area_url_add_page_count)
                if checkHttpStatusCode(r, search_area_url_add_page_count):
                    
                    # 検索店数の更新
                    if page_count == loop_page_count - 1:
                        search_store_count = Izakaya.search_store_count - Izakaya.separator_page_count * page_count
                    else:
                        search_store_count = 20
                    
                    getPersonalStoreInfo(r, search_store_count)
                    
                # 検索エリアURL + ページ数のHTTP情報が不正
                else:
                    return False
            
        # 検索エリアURLのスクレイピング店数取得結果が0
        else:
            return False
    
    # 検索エリアURLのHTTP情報が不正
    else:
        return False
    
    return True
        
class Izakaya:
    
    columns_izakaya = ['store_id'
                     , 'store_name'
                     , 'store_genre'
                     , 'drink_name'
                     , 'drink_price'
                     , 'store_distance'
                     , 'store_budget'
                     , 'store_evaluation'
                     , 'store_url'
                     , 'error_flag'
                     , 'error_type']
    
    df_izakaya = pd.DataFrame(columns=columns_izakaya)
    
    # 区切りページ数
    separator_page_count = 20
    
    # 検索エリアURL
    search_area_url = "https://tabelog.com/chiba/A1202/A120202/R9599/rstLst/izakaya/"

    # 検索ワード
    search_words = ["ビール", "モルツ", "ドライ", "ラガー", "黒ラベル"]

    # 検索店数
    search_store_count = 3
    
    # カラム：個別店ID
    personal_store_id = 0

    def __init__(self, area_store_url, search_store_count):
        
        if getAreaStoreInfo(area_store_url, search_store_count):
            display(Izakaya.df_izakaya)
            # display(Izakaya.df_izakaya.sort_values(by='drink_price'))
            Izakaya.df_izakaya.to_csv("./Izakaya_MinamiGyoutoku.csv", encoding='utf_8_sig')
        

# アプリ「Izakaya」実行
Izakaya(Izakaya.search_area_url, Izakaya.search_store_count)


AttributeError: 'NoneType' object has no attribute 'text'